# 回答結果を数値に変換する
処理方法  
- 性別：0=男性，1=女性
- 年代：0=10代，1=20代，…
- 遊びのジャンル：0=アウトドア，1=エンタメ，2=スポーツ，3=観光，4=遊園地，5=飲み会，6=買い物，7=ショー，8=未回答
- その他の質問：回答の連番（「1: ～～」の「1」の部分）-1の値

# import・定数

In [76]:
import numpy as np
import pandas as pd

In [77]:
# 回答結果
# input_path = '/home/res/answers/questionnaire_latest.csv'
input_path = '/home/res/answers/answers_2020-08-13-0918.csv'
output_path = '/home/res/converted/answers_int.csv'
question_list_path = '/home/res/converted/question_list.csv'
genre_list_path = '/home/res/converted/genre_list.csv'

# CSV読み込み

In [78]:
# 回答結果を読み込み
df_csv = pd.read_csv(input_path)

In [79]:
# 明後日の予定の質問の選択肢の連番が4と5で逆になってしまっているので修正
def map_change_num(x):
    if x == '5 : だいたい予定が詰まっている (25%)':
        x = '4 : だいたい予定が詰まっている (25%)'
    elif x == '4 : 朝から外せない予定がある (0%)':
        x = '5 : 朝から外せない予定がある (0%)'
    return x

df_csv.iloc[:, 17] = df_csv.iloc[:, 17].map(map_change_num)

In [80]:
# 複数回答の区切り文字が";"と", "で２種類あるので", "に統一
df_csv.iloc[:, 6] = df_csv.iloc[:, 6].map(lambda x: x.replace(';', ', ') if type(x) is str else x)

In [81]:
# 遊びのジャンルのリスト（未回答は"なし"に変換）
lst_genre = ['アウトドア（マリンスポーツ・釣り・BBQ・登山・ハイキングなど）', 
             'エンタメ（ボウリング場・カラオケ・ゲームセンターなど）', 
             'スポーツ（ボルダリング・ゴルフ・サイクリング・ジム・ヨガなど）',
             '観光（観光名所巡り・日帰り温泉・食べ歩きなど）',
#              '遊園地・動物園・水族館・海水浴場',
             '友達との飲み会（外飲み・飲み歩きなど）',
             '外での買い物（アウトレットなど）',
             'ショー・展示・催し（映画館・ミュージカル・博物館・美術館など）',
             'なし']

In [82]:
# 遊園地と回答した結果を削除
df_csv = df_csv.drop(df_csv.index[
    (df_csv.iloc[:, 3] == '遊園地・動物園・水族館・海水浴場') | 
    (df_csv.iloc[:, 4] == '遊園地・動物園・水族館・海水浴場') |
    (df_csv.iloc[:, 5] == '遊園地・動物園・水族館・海水浴場') |
    (df_csv.iloc[:, 18] == '遊園地・動物園・水族館・海水浴場')
]).reset_index(drop=True)

In [83]:
# 複数回答の列を抽出（nanを"なし"に変換）
df_csv.iloc[:, 6] = df_csv.iloc[:, 6].fillna('なし').map(lambda x: x.split(', '))

In [84]:
# 複数回答の中に遊園地が含まれていたら行ごと削除
df_csv = df_csv.drop(index=[i for i, s in enumerate(df_csv.iloc[:, 6]) if '遊園地・動物園・水族館・海水浴場' in s])

# カラムを数値に変換

In [60]:
# カラムを質問文から連番に変換
questions = pd.factorize(df_csv.columns)
df_csv.columns = questions[0]

# 性別を数値に変換

In [61]:
# 性別を数値に変換
genders = pd.factorize(df_csv.iloc[:, 1])
df_csv.iloc[:, 1] = genders[0]

# ジャンルを数値に変換

In [62]:
# 遊びのジャンルを数値に変換
for i in [3, 4, 5, 18]:
    df_csv.iloc[:, i] = df_csv.iloc[:, i].fillna('なし').map(lambda x: lst_genre.index(x))

In [63]:
# 複数回答の遊びのジャンルを数値に変換
def map_stoi(lst_ans):
    lst_ans_int = []
    for ans in lst_ans:
        lst_ans_int.append(lst_genre.index(ans))
    return lst_ans_int
    
df_csv.iloc[:, 6] = df_csv.iloc[:, 6].map(map_stoi)

# その他の質問を数値に変換

In [64]:
# この時点で未回答の質問があるのはアウトなので除外
df_csv = df_csv.dropna()

In [65]:
# 連番付きの質問の回答を数値に変換
for i in [2] + list(range(7, 18)) + list(range(19, len(df_csv.columns))):
    df_csv.iloc[:, i] = df_csv.iloc[:, i].map(lambda x: int(x[0]) - 1)

# CSV出力

In [66]:
df_csv.columns = questions[1]

In [67]:
df_csv

,タイムスタンプ,あなたの性別は？,あなたの年代は？,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，最も当てはまるものを1つ選択してください(必須)．,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，2番目に当てはまるものを1つ選択してください(任意)．,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，3番目に当てはまるものを1つ選択してください(任意)．,その他，普段よく遊ばれるジャンルがあれば，当てはまるものを複数選択してください．,上記で選択した遊び(普段よくする遊び)を行うときは，1日でお金を平均で幾らほど使われますか？,上記で選択した遊び(普段よくする遊び)を行うときは，自然に囲まれた中で遊びたい気分のときが多いですか？それとも街や駅前に出て遊びたい気分のときが多いですか？（%の値は～したいという気持ちの度合いです）,上記で選択した遊び(普段よくする遊び)を行うときは，エキサイティングな遊びをしたい気分のときが多いですか？それとも，ゆったりのんびり遊びたい気分のときが多いですか？（%の値は～したいという気持ちの度合いです）,...,現在，身体的に疲れを感じていますか？（%の値は疲労の度合いです）,現在，精神的に疲れを感じていますか？（%の値は疲労の度合いです）,明後日に予定(会社や大学などの外せない予定)はありますか？（%の値は予定の無さ具合です）,あなたは明日遊びに行くとするならば，どの遊びを選択しますか？次のジャンルのうち，明後日の予定や現在の身体的・精神的状態を考慮した後，当てはまるものを1つ選択してください．,通常，自分から話を始めることはない,どちらかというと控えめで，静かな性格である,深く考えるより，すぐに行動したい,リーダーとして皆を引っ張っていきたい,近くでイベントがあれば参加するほうだ,どちらかというと，開放的で，陽気なタイプだと思う
0,2020/08/10 3:36:39 午前 GMT+9,0,1,1,3,0,"[0, 1, 3, 4, 5]",3,3,2,...,3,3,3,1,1,2,4,2,3,3
1,2020/08/10 8:35:24 午前 GMT+9,1,1,6,3,5,[4],3,1,1,...,5,5,4,6,2,4,2,3,1,0
2,2020/08/10 9:12:24 午前 GMT+9,0,1,5,4,2,"[2, 4, 5]",5,3,3,...,4,4,4,5,3,4,3,0,3,3
3,2020/08/10 10:19:01 午前 GMT+9,1,1,1,0,3,[7],1,4,2,...,4,4,2,5,4,2,5,0,0,3
4,2020/08/10 1:03:58 午後 GMT+9,0,1,5,4,3,[1],1,3,3,...,3,3,2,5,1,3,2,3,3,2
5,2020/08/10 2:24:29 午後 GMT+9,0,1,1,2,5,"[3, 4]",0,3,2,...,2,1,0,1,4,5,3,0,3,0
6,2020/08/10 3:51:34 午後 GMT+9,0,1,4,7,7,[7],0,3,1,...,5,5,0,6,2,1,4,1,1,0
7,2020/08/10 4:35:27 午後 GMT+9,0,1,6,4,4,[7],3,4,5,...,4,4,1,4,5,4,5,1,4,0
8,2020/08/10 4:46:50 午後 GMT+9,0,2,4,3,1,"[0, 5, 6]",5,3,3,...,4,4,4,4,2,4,3,1,3,4
9,2020/08/10 4:53:59 午後 GMT+9,0,1,1,4,0,[7],1,3,1,...,3,4,4,4,3,5,3,0,1,1


In [43]:
# 数値情報に変換した回答結果を保存
df_csv.to_csv(output_path, index=False)

In [61]:
# 質問文一覧
df_questions = pd.DataFrame(list(zip(questions[0], questions[1])), columns=['no', 'question'])
df_questions.to_csv(question_list_path, index=False)

df_questions

,no,question
0,0,タイムスタンプ
1,1,あなたの性別は？
2,2,あなたの年代は？
3,3,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，最も当てはまるものを1...
4,4,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，2番目に当てはまるもの...
5,5,あなたは普段，どのような遊びをよくされていますか？次のジャンルのうち，3番目に当てはまるもの...
6,6,その他，普段よく遊ばれるジャンルがあれば，当てはまるものを複数選択してください．
7,7,上記で選択した遊び(普段よくする遊び)を行うときは，1日でお金を平均で幾らほど使われますか？
8,8,上記で選択した遊び(普段よくする遊び)を行うときは，自然に囲まれた中で遊びたい気分のときが多...
9,9,上記で選択した遊び(普段よくする遊び)を行うときは，エキサイティングな遊びをしたい気分のとき...


In [62]:
# ジャンル一覧
df_genre = pd.DataFrame(list(zip(list(range(len(lst_genre))), lst_genre)), columns=['no', 'genre'])
df_genre.to_csv(genre_list_path, index=False)

df_genre

,no,genre
0,0,アウトドア（マリンスポーツ・釣り・BBQ・登山・ハイキングなど）
1,1,エンタメ（ボウリング場・カラオケ・ゲームセンターなど）
2,2,スポーツ（ボルダリング・ゴルフ・サイクリング・ジム・ヨガなど）
3,3,観光（観光名所巡り・日帰り温泉・食べ歩きなど）
4,4,遊園地・動物園・水族館・海水浴場
5,5,友達との飲み会（外飲み・飲み歩きなど）
6,6,外での買い物（アウトレットなど）
7,7,ショー・展示・催し（映画館・ミュージカル・博物館・美術館など）
8,8,なし
